In [1]:
import pyspark.sql.functions as F

StatementMeta(, 71467d8b-3d67-4217-a4ed-1ed311b44faf, 3, Finished, Available, Finished)

In [2]:
# Cargar datos y resultados de clustering
df_clusters = spark.table("Predicciones_Clusters")  # numContrato, idCliente, cluster
df_original = spark.sql("SELECT * FROM lakehouseSilver.dfcontratosfinal")

StatementMeta(, 71467d8b-3d67-4217-a4ed-1ed311b44faf, 4, Finished, Available, Finished)

✅ Datos combinados para interpretación.


In [ ]:
# Unir para análisis

df_union = df_clusters.join(df_original, on="numContrato", how="left")

print("✅ Datos combinados para interpretación.")

In [3]:
# Calcular promedios de variables numéricas por cluster

numeric_cols = [
    "montoSaldoAnterior","montoLeasing","plazoOriginal","plazo","tasa",
    "montoDeposito","montoDepositoAbonado","montoSinSeguro","montoSeguroContrato",
    "montoMantenimiento","montoSeguroMantenimiento","montoCompraFinal",
    "montoProveedor","montoValorFiscal","diasGraciaPrimerPago","montoiva"
]

agg_exprs = [F.avg(c).alias(f"avg_{c}") for c in numeric_cols]
stats_clusters = df_union.groupBy("cluster").agg(*agg_exprs)


StatementMeta(, 71467d8b-3d67-4217-a4ed-1ed311b44faf, 5, Finished, Available, Finished)

In [4]:
# Distribución de variables categóricas por cluster

categorical_cols = ["Canal", "Industria", "Segmento", "valorCliente"]

dist_exprs = []
for cat in categorical_cols:
    dist = df_union.groupBy("cluster", cat).count()
    dist_exprs.append(dist)

# Se procede a guardarla distribución de categóricas en tablas separadas

for cat, dist in zip(categorical_cols, dist_exprs):
    table_name = f"Distribucion_{cat}_Cluster"
    dist.write.format("delta").mode("overwrite").saveAsTable(table_name)
    print(f"✅ Distribución por {cat} guardada en {table_name}")


StatementMeta(, 71467d8b-3d67-4217-a4ed-1ed311b44faf, 6, Finished, Available, Finished)

✅ Distribución por Canal guardada en Distribucion_Canal_Cluster
✅ Distribución por Industria guardada en Distribucion_Industria_Cluster
✅ Distribución por Segmento guardada en Distribucion_Segmento_Cluster
✅ Distribución por valorCliente guardada en Distribucion_valorCliente_Cluster


In [5]:
# Guardar tabla de resumen numérico

stats_clusters.write.format("delta").mode("overwrite").saveAsTable("ResumenClusters")

print("✅ Tabla 'ResumenClusters' creada con promedios por cluster.")


StatementMeta(, 71467d8b-3d67-4217-a4ed-1ed311b44faf, 7, Finished, Available, Finished)

✅ Tabla 'ResumenClusters' creada con promedios por cluster.
